# December 24, 2023

https://adventofcode.com/2023/day/24

In [240]:
import numpy as np
from sympy import factorint, divisors

In [15]:
def parse_input(text):
    init = list()
    vel = list()
    for line in text:
        one, two = line.split(" @ ")
        init.append( [ int(x) for x in one.split(", ") ] )
        vel.append( [ int(x) for x in two.split(", ") ] )
    return {"init":init, "vel":vel}

In [16]:
text = f'''19, 13, 30 @ -2,  1, -2
18, 19, 22 @ -1, -1, -2
20, 25, 34 @ -2, -2, -4
12, 31, 28 @ -1, -2, -1
20, 19, 15 @  1, -5, -3'''

test = parse_input(text.split("\n"))

In [17]:
fn = "data/24.txt"
with open(fn, "r") as file:
    text = file.readlines()
puzz = parse_input(text)

### Part 1

In [18]:
test

{'init': [[19, 13, 30],
  [18, 19, 22],
  [20, 25, 34],
  [12, 31, 28],
  [20, 19, 15]],
 'vel': [[-2, 1, -2], [-1, -1, -2], [-2, -2, -4], [-1, -2, -1], [1, -5, -3]]}

In [82]:
def linear_eq( p0, v0, p1, v1, eps=1e-12 ):

    # check for parallel lines:
    if (v1[1]/v0[1] == v1[0]/v0[0]):
        return None
        
    # determine time and x and y coordinates where the paths intersect
    x = ( (p0[1] - p1[1]) + ( p1[0]*v1[1]/v1[0] - p0[0]*v0[1]/v0[0] ) ) / (v1[1]/v1[0] - v0[1]/v0[0] )

    t0 = (x - p0[0]) / v0[0]
    t1 = (x - p1[0]) / v1[0]

    y = p0[1] + v0[1]*t0
    
    # check intersection: y's should match too
    #y1 = p1[1] + v1[1]*t1
    #if abs(y-y1) > eps:
    #    print( p0, v0, p1, v1, t0, t1, x, y)
    #    raise Exception("intersection check fail")
    #assert abs(y-y1) < eps

    return t0, t1, x, y    

In [83]:
def part1(puzz, xlim, ylim, verbose = False):
    pvec = puzz["init"]
    vvec = puzz["vel"]
    
    smorsh = 0
    for i, p0 in enumerate(pvec):
        if verbose:
            print(i)
        v0 = vvec[i]
        for j in range(i+1, len(pvec), 1):
            p1 = pvec[j]
            v1 = vvec[j]

            intersection = linear_eq( p0, v0, p1, v1 )
            if intersection is None:
                continue
            t0, t1, x, y = intersection
            if ( t0 >= 0 and t1 >= 0 and x >= xlim[0] and x <= xlim[1] and y >= ylim[0] and y <= ylim[1] ):
                smorsh += 1
                if verbose:
                    print(f"SMORSH! {i}, {j}")

    return smorsh



In [84]:
part1(test, [7,27], [7,27], verbose=True )

0
SMORSH! 0, 1
SMORSH! 0, 2
1
2
3
4


2

In [86]:
part1(puzz, [200000000000000, 400000000000000], [200000000000000, 400000000000000])

15889

### Part 2

In [185]:
# some functions for getting all factors of an integer

def prime_factors(x, known={1:[]}):
    if x in known:
        return known[x]

    primes = [2,3,5,7,11,13,17,19,23,29,31,37]
    for p in primes:
        if x % p == 0:
            pf = [p] + prime_factors( int(x/p) )
            known[x] = pf
            return known[x]

    known[x] = [x]
    return known[x]

def factors(x):
    if x == 1:
        return [1]

    pf = prime_factors(x)
    return _all_products( pf )

def _all_products(pf):
    if len(pf) == 1:
        return set([1,pf[0]])

    facs = _all_products(pf[1:])
    facs |= set([pf[0]*f for f in facs])
    return facs



In [283]:
def possible_speeds( pos0, pos1, vel0 ):
    '''if two hailstones have the same speed, it puts constraints on projectile speed'''

    # if rock hits further (from 0) hailstone first, then vel0 - vrock must be a factor of |pos1-pos0|
    # this is because the other hailstone must catch up
    # and it does so at a rate of vel0 - vrock per nanosecond

    # if rock hits closer (to 0) hailstone first, then vrock - vel0 must be a factor of |pos1-pos|
    # same reason, but the rock is catching up to the hailstone
    delta = abs(pos1 - pos0)
    facs = divisors(delta)
    poss = set( [vel0 + f for f in facs] + [vel0 - f for f in facs])

    # if vel0 < 0:
    #     pass
    #     print("negating")
    #     print(poss)
    #     poss = set( [ -vel for vel in poss ] )
    #     print(poss)
    
    return poss

In [356]:
def determine_speed( pvec, vvec ):
    poss = set()
    order = np.argsort( vvec )

    for i, idx in enumerate(order[:-1]):
        j = i + 1
        jdx = order[j]
        
        # keep trying the next position and see if it's a matching speed
        while (j < len(order) and vvec[jdx] == vvec[idx]):
            #print(f'''Considering {idx} ({pvec[idx]}; {vvec[idx]}) and {jdx} ({pvec[jdx]}; {vvec[jdx]})''')
            new_poss = possible_speeds( pvec[idx], pvec[jdx], vvec[idx] )
            #print("new poss:", new_poss)
            if len(poss) == 0:
                poss = new_poss
            else:
                poss &= new_poss

            #print("updated poss:", poss)
            if len(poss) == 0:
                raise BaseException("I don't know what I'm doing")
            if len(poss) == 1:
                return poss.pop()

            j += 1
            jdx = order[j]

    print("Warning: Multiple speeds are possible...")
    return poss

def determine_pos( pvec, vvec, vrock ):
    idx = 285
    jdx = 197
    idx, jdx = 0, 1
    a = np.array( [ [1, 0, 0, vrock[0]-vvec[idx][0], 0],
                    [0, 1, 0, vrock[1]-vvec[idx][1], 0],
                    [0, 0, 1, vrock[2]-vvec[idx][2], 0],
                    [1, 0, 0, 0, vrock[0]-vvec[jdx][0]],
                    [0, 1, 0, 0, vrock[1]-vvec[jdx][1]]
    ])


    b = np.array( [pvec[idx][0], pvec[idx][1], pvec[idx][2], pvec[jdx][0], pvec[jdx][1]] )

    coefs = np.linalg.solve(a, b) # rounding errors?
    print("x0, y0, z0, t1, t2:", *coefs)

    final = [ int(np.round(x)) for x in coefs[:3]]
    return final

In [357]:
def part2( puzz ):
    # determine xspeed:
    px = [ p[0] for p in puzz["init"] ]
    vx = [ p[0] for p in puzz["vel"] ]
    dx = determine_speed( px, vx )

    py = [ p[1] for p in puzz["init"] ]
    vy = [ p[1] for p in puzz["vel"] ]
    dy = determine_speed( py, vy )

    pz = [ p[2] for p in puzz["init"] ]
    vz = [ p[2] for p in puzz["vel"] ]
    dz = determine_speed( pz, vz )

    print(f'''Rock velocity: @ {dx}, {dy}, {dz}''')

    x0, y0, z0 = determine_pos( puzz["init"], puzz["vel"], (dx, dy, dz) )
    print(f'''\nInit position: {x0}, {y0}, {z0}''')

    return x0 + y0 + z0

In [358]:
part2(puzz)

Rock velocity: @ 136, -237, 110
x0, y0, z0, t1, t2: 181540669791003.88 404991404832784.06 214854400593114.3 751758998126.0012 514753497469.0005

Init position: 181540669791004, 404991404832784, 214854400593114


801386475216902